# Otimização de Hiperparâmetros para Detecção de Fraude em Cartões de Crédito

## Instalação de Dependências (se necessário)

In [ ]:
# ! pip install scikit-optimizer
# ! pip install imblearn

## Importação de Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from imblearn.over_sampling import SMOTE
from joblib import dump

## Carregamento e Pré-processamento dos Dados

In [ ]:
# Carregar o dataset
df = pd.read_csv('../data/creditcard.csv')

In [ ]:
# Escalar as colunas 'Amount' e 'Time'
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df['Time'] = scaler.fit_transform(df['Time'].values.reshape(-1, 1))

In [ ]:
# Separar features (X) e target (y)
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Balanceamento de Dados com SMOTE

In [ ]:
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_train, y_train)

## Definição do Espaço de Hiperparâmetros

In [ ]:
param_space = {
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'min_child_weight': Integer(1, 10),
    'max_depth': Integer(3, 10),
    'gamma': Real(0, 1),
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.5, 1.0),
    'scale_pos_weight': Real(1, 100),
    'n_estimators': Integer(50, 500),
    'reg_alpha': Real(0.0001, 1, prior='log-uniform'),
    'reg_lambda': Real(0.0001, 1, prior='log-uniform')
}

## Configuração do Modelo XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',
    random_state=42,
    n_jobs=-1
)

## Definição da Função de Avaliação Personalizada

In [ ]:
def custom_score(model, X, y):
    y_pred = model.predict_proba(X)[:, 1]
    return average_precision_score(y, y_pred)

## Otimização Bayesiana dos Hiperparâmetros

In [ ]:
bayes_search = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=param_space,
    n_iter=50,
    cv=5,
    scoring=custom_score,
    verbose=1,
    random_state=42,
    return_train_score=True
)
bayes_search.fit(X_resampled, y_resampled)

## Avaliação do Melhor Modelo

In [ ]:
best_params = bayes_search.best_params_
print("Melhores hiperparâmetros encontrados:")
for param, value in best_params.items():
    print(f"{param}: {value}")

In [ ]:
best_model = bayes_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

In [ ]:
auc = roc_auc_score(y_test, y_pred_proba)
ap = average_precision_score(y_test, y_pred_proba)
print(f"AUC no conjunto de teste: {auc:.4f}")
print(f"Average Precision no conjunto de teste: {ap:.4f}")

## Treinamento Final com Dados Balanceados

In [ ]:
X_resampled_final, y_resampled_final = SMOTE(random_state=42).fit_resample(X, y)
final_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',
    random_state=42,
    n_jobs=-1,
    **best_params
)
final_model.fit(X_resampled_final, y_resampled_final)

## Salvando o Modelo e os Hiperparâmetros

In [ ]:
dump(final_model, '../models/final_model.joblib')
dump(best_params, '../models/best_params.joblib')
print("Modelo e hiperparâmetros salvos com sucesso!")